In [ ]:
pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.2 MB/s eta 0:00:00


In [ ]:
from Crypto.PublicKey import RSA

# Генерація ключів
key = RSA.generate(2048)

# Збереження приватного ключа
private_key = key.export_key()
with open("private.pem", "wb") as f:
    f.write(private_key)

# Збереження публічного ключа
public_key = key.publickey().export_key()
with open("receiver.pem", "wb") as f:
    f.write(public_key)

print("Ключі згенеровано та збережено в файлах private.pem та receiver.pem.")

Ключі згенеровано та збережено в файлах private.pem та receiver.pem.


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP

# Дані для шифрування
data = "Секретне повідомлення: Це тест гібридної криптосистеми.".encode("utf-8")

# Імпорт публічного ключа отримувача
recipient_key = RSA.import_key(open("receiver.pem").read())

# Генерація сесійного ключа AES (16 байт для AES-128)
session_key = get_random_bytes(16)

# Шифрування сесійного ключа за допомогою RSA з OAEP
cipher_rsa = PKCS1_OAEP.new(recipient_key)
enc_session_key = cipher_rsa.encrypt(session_key)

# Шифрування даних за допомогою AES в режимі EAX
cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(data)

# Збереження зашифрованих даних у файл
with open("encrypted_data.bin", "wb") as f:
    [f.write(x) for x in (enc_session_key, cipher_aes.nonce, tag, ciphertext)]

print("Дані зашифровано та збережено в encrypted_data.bin.")

Дані зашифровано та збережено в encrypted_data.bin.


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP

# Імпорт приватного ключа
private_key = RSA.import_key(open("private.pem").read())

# Читання зашифрованих даних з файлу
with open("encrypted_data.bin", "rb") as f:
    enc_session_key = f.read(private_key.size_in_bytes())
    nonce = f.read(16)
    tag = f.read(16)
    ciphertext = f.read(-1)

# Дешифрування сесійного ключа за допомогою RSA з OAEP
cipher_rsa = PKCS1_OAEP.new(private_key)
session_key = cipher_rsa.decrypt(enc_session_key)

# Дешифрування даних за допомогою AES в режимі EAX
cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce=nonce)
data = cipher_aes.decrypt_and_verify(ciphertext, tag)

print("Розшифровані дані:", data.decode("utf-8"))

Розшифровані дані: Секретне повідомлення: Це тест гібридної криптосистеми.
